In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import json
import pickle
import os
import glob
from sklearn.feature_extraction.text import CountVectorizer
from kiwipiepy import Kiwi
from bertopic import BERTopic

ModuleNotFoundError: No module named 'bertopic'

In [2]:
data_path = './data/빅카인즈_크롤링.xlsx'
text_df = pd.read_excel(data_path, index_col=False)
text_df.reset_index(drop=True, inplace=True)

In [3]:
text_df

,Date,Title,Content
0,2023-11-27,[사설] 생활권 침해 없는데 봉안당 불허···님비에 법원 제동,극단적인 지역이기주의에 지역 현안들이 발목이 잡혀있는 가운데 법원이 과도한 님비현상...
1,2023-11-27,[전남일보]기고·김정화>지방소멸 대책 신안에 있다,김정화 신안군농업기술센터소장\n김정화 소장\n“바보야! 문제는 경제야!”(“It’s...
2,2023-11-27,“학생 수 감소했다면 학교용지비용 부담 필요 없어”,[KBS 광주]\n취학 인구가 감소했는데도 학교용지부담금을 부과한 행정처분은 부당하...
3,2023-11-27,서삼석 예결위원장 “올해 전 부처가 증액 요구…특활비 무조건 삭감보다 투명성 제고”...,서삼석 국회 예산결산특별위원회 위원장이 17일 오후 서울 여의도 국회에서 헤럴드경제...
4,2023-11-27,"경단녀서 꽃가게 사장님으로… ""지방은 기회의 공간"" [지방시대 지방영웅]",<4> 전남 목포 '두리하나 플라워' 임효백씨 아이 때문에 결심한 '지방살이'서 꿈...
...,...,...,...
4230,2021-01-03,'포스트 코로나 원년' 올해는 꼭 보여주고 싶 '소',■미리보는 2021 3대 비엔날레\n광주비엔날레에 전시되는 아나 마리아 밀란 작 '...
4231,2021-01-03,인사 종합,[인사]광주시\n◇3급 전보△자치행정국장 김일융 △공무원교육원장 김애리 △서구 부구...
4232,2021-01-03,"김종식 목포시장 ""호랑이눈·소 걸음으로 뚝심있게 계획 실행""","김종식 목포시장1\n김종식 목포시장은 신축년 새해를 맞아 시정 목표와 관련해 ""코로..."
4233,2021-01-02,[신년사] 김종식 목포시장,김종식 목포시장\n존경하는 시민 여러분! 그리고 동료 공직자 여러분! 2021년 새...


In [4]:
class CustomTokenizer:
    def __init__(self, kiwi):
        self.kiwi = kiwi
    def __call__(self, text):
        result = list()
        for token in self.kiwi.tokenize(text):
            if token[1] in ["NNG", "NNP", "NNB", "NR", "NP"] and int(token[3]) > 1:
                result.append(token[0])
        return result

https://maartengr.github.io/BERTopic/api/bertopic.html#bertopic._bertopic.BERTopic

In [5]:
stopwords = ['의원', '위원', '위원장', '과장', '시장', '때문', '지난해', '의회', '기자', '지역', '지원', '사업', '문화', '전남', '무안', '무안군', '목포', '목포시', '신안', '신안군', '광주']

mytokenizer = CustomTokenizer(Kiwi())
vectorizer =  CountVectorizer(tokenizer=mytokenizer, max_features=300, stop_words=stopwords)

model = BERTopic(embedding_model="paraphrase-multilingual-MiniLM-L12-v2", \
		vectorizer_model=vectorizer,
        nr_topics=10, # 문서를 대표하는 토픽의 갯수
        top_n_words=10,
        calculate_probabilities=True)

In [6]:
topics, probs = model.fit_transform(text_df['Content'])

In [40]:
model.get_topic(-1)

[('교육', 0.044094572856069894),
 ('추진', 0.025294009785808154),
 ('운영', 0.025225237682737055),
 ('주민', 0.024579255504268073),
 ('센터', 0.02367393077784496),
 ('학교', 0.02327524150550635),
 ('청년', 0.023184841458612302),
 ('전국', 0.023171640776891468),
 ('진행', 0.022351629624797694),
 ('관광', 0.022180952726775935)]

In [7]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1488,-1_교육_추진_운영_주민,"[교육, 추진, 운영, 주민, 센터, 학교, 청년, 전국, 진행, 관광]",[전국 최초 목포문학박람회 개최 등 관광·문화·사회경제 전반 효과 기대 설날 앞두고...
1,0,753,0_교육_학교_학생_제공,"[교육, 학교, 학생, 제공, 운영, 대상, 사회, 참여, 진행, 건강]",[●김대중 전남도교육감 취임 1주년 기자회견\n김대중 전남도교육감이 4일 무안군 도...
2,1,550,1_도시_시민_추진_교육,"[도시, 시민, 추진, 교육, 해양, 청년, 문학, 청소년, 통합, 운영]","[목포시가 신재생에너지산업과 수산식품산업, 관광산업 등 ‘3대 미래전략산업 육성’사..."
3,2,400,2_후보_농협_선거_지방,"[후보, 농협, 선거, 지방, 발전, 대표, 교육, 산업, 군수, 농업]",[박우량 전남 신안군수/사진=신안군청 제공\n[머니투데이 더리더 편승민 송민수 더리...
4,3,341,3_갯벌_해양_조성_생물,"[갯벌, 해양, 조성, 생물, 환경, 단지, 지구, 시설, 생태, 선정]","[무안·영광 먹거리, 완도 힐링, 여수 新 경험, 순천 생태, 나주 반려동물 특화자..."
5,4,326,4_인구_안전_발생_예방,"[인구, 안전, 발생, 예방, 학교, 청년, 시설, 병원, 점검, 코로나]",[전남 목포시 해상 케이블카 정기점검에 나선 교통안전공단 직원들이 안전점검을 마친 ...
6,5,191,5_관광_마을_세계_홍어,"[관광, 마을, 세계, 홍어, 전시, 체험, 축제, 갯벌, 프로그램, 한국]","[압해도 송공항에서 바라보는 천사대교는 끝이 보이지 않는다. 교량길이만 7,220m..."
7,6,137,6_지방_인구_소멸_예산,"[지방, 인구, 소멸, 예산, 감소, 지자체, 재단, 대응, 계획, 정부]",[인구가 줄어 소멸 위기에 놓인 자치단체를 지원하기 위한 첫 '지방소멸대응기금' 배...
8,7,34,7_농가_생산_양식_홍보,"[농가, 생산, 양식, 홍보, 소득, 식품, 우수, 국제, 안정, 축제]",[전남 신안군이 지역 특산 개체굴인 '1004굴' 수출을 위해 오는 15~20일 아...
9,8,15,8_장애인_체험_센터_예산,"[장애인, 체험, 센터, 예산, 설립, 학생, 공모, 활용, 체육, 조성]",[이상일 용인시장.\n[헤럴드경제(용인)=박정규 기자]용인특례시(시장 이상일)가 대...


In [30]:
model.visualize_topics(topics=list(range(-1,9)))

In [9]:
model.visualize_distribution(probs[-1])

In [31]:
for i in range(-1, 9):
    print(i,'번째 토픽 :', model.get_topic(i))

-1 번째 토픽 : [('교육', 0.044094572856069894), ('추진', 0.025294009785808154), ('운영', 0.025225237682737055), ('주민', 0.024579255504268073), ('센터', 0.02367393077784496), ('학교', 0.02327524150550635), ('청년', 0.023184841458612302), ('전국', 0.023171640776891468), ('진행', 0.022351629624797694), ('관광', 0.022180952726775935)]
0 번째 토픽 : [('교육', 0.06775955603427407), ('학교', 0.040817979981972334), ('학생', 0.0369819360094166), ('제공', 0.034670281063559204), ('운영', 0.030777224102877976), ('대상', 0.029728399583389156), ('사회', 0.028401491147864038), ('참여', 0.02834324548427738), ('진행', 0.027937483005130395), ('건강', 0.027379174146816426)]
1 번째 토픽 : [('도시', 0.0498617841398647), ('시민', 0.04020547533223901), ('추진', 0.03749799406265616), ('교육', 0.034591744142847304), ('해양', 0.03373666970822963), ('청년', 0.03370385480723756), ('문학', 0.03339687289402154), ('청소년', 0.03311056511802926), ('통합', 0.032497505969755404), ('운영', 0.031209851802820897)]
2 번째 토픽 : [('후보', 0.06025015310539677), ('농협', 0.04103165442902919), ('선거', 0.0

In [39]:
model.visualize_heatmap(topics=list(range(-1,8)))

In [14]:
timestamps = text_df['Date'].to_list()
content_text = text_df['Content'].to_list()

topics_over_time = model.topics_over_time(content_text, timestamps)

2023-12-26 01:40:23,170 - BERTopic - WARNING: There are more than 100 unique timestamps (i.e., 915) which significantly slows down the application. Consider setting `nr_bins` to a value lower than 100 to speed up calculation. 


In [26]:
model.visualize_topics_over_time(topics_over_time, topics=list(range(-1,8)))

문서 예측

In [ ]:
new_doc = docs[0]
print(new_doc)

In [ ]:
topics, probs = model.transform([new_doc])
print('예측한 토픽 번호 :', topics)

모델 저장

In [ ]:
model.save("my_topics_model")

불러오기

In [ ]:
BerTopic_model = BERTopic.load("my_topics_model")